# 데이터 재구성

- kogpt2 + 한국어 데이터

- 정제된 동화 데이터 최소 5000~10000개 필요 → 데이터 불리기, 장르는 라벨링

- 기승전결 문단으로 데이터 재구성하여 학습 (문장 개수 단위, 장르는 라벨링 후에 prefix로 붙일 예정)

## EDA

In [1]:
df1 = open('동화_본문_합본/동화본문_211223수정.txt', 'r').read()
df2 = open('tale.txt', 'r').read()
df = []
except_idx = []
title = []
for i, sentence in enumerate(df1.split('\n\n\n')):
    try:
        story = sentence.split('\n\n')[1].split('\n')
        title.append([i+1, sentence.split('\n\n')[0]])
    except:
        except_idx.append(i)
        story = sentence.split('\n')[2:]
        title.append([i+1, sentence.split('\n')[1]])
    df.extend(story)
for i in df2.split('\n\n'):
    story = i.split('\n')
    df.extend(story)
len(df)  # 한국어 동화 데이터 문장 개수

55211

In [184]:
print('1,2차 합본 동화 개수 :', len(df1.split('\n\n\n')))
print('tale 동화 개수 :', len(df2.split('\n\n')))

1,2차 합본 동화 개수 : 822
tale 동화 개수 : 227


In [187]:
except_idx

[41, 44, 47, 90]

In [156]:
import pandas as pd

title_txt = pd.DataFrame(title, columns=['ID', '코딩 소제목'])
title_txt['코딩 소제목'] = title_txt['코딩 소제목'].str.lstrip('\n')
title_txt

,ID,코딩 소제목
0,1,방귀 대장은 누구?
1,2,송아지와 바꾼 무
2,3,낫 놓고 기역 자도 모르는 양반
3,4,내기 장기
4,5,꿀 똥 싸는 강아지
...,...,...
817,818,수다 부인
818,819,나라를 세운 단군
819,820,알에서 태어난 주몽
820,821,호랑이 잡는 기왓장


In [157]:
title_xlsx = pd.read_excel('동화_본문_합본/동화제목_211223수정.xlsx').iloc[:824,:2]
title_xlsx

,ID,코딩 소제목
0,1,방귀 대장은 누구?
1,2,송아지와 바꾼 무
2,3,낫 놓고 기역 자도 모르는 양반
3,4,내기 장기
4,5,꿀 똥 싸는 강아지
...,...,...
819,820,수다 부인
820,821,나라를 세운 단군
821,822,알에서 태어난 주몽
822,823,호랑이 잡는 기왓장


In [158]:
set(title_xlsx['코딩 소제목']) - set(title_txt['코딩 소제목'])

{'내 친구 개', '내가 도와줄게!', '밤의 소리를 들어 봐', '호랑이와 밤송이 형님'}

In [178]:
set(title_txt['코딩 소제목']) - set(title_xlsx['코딩 소제목'])

{'김만중', '쌀이 나오는 바가지'}

## Preprocessing

10 sentence per input (성별 switch)

In [140]:
# 문장 10개당 하나의 string으로, 5문장씩 겹쳐서 구성
df1 = open('동화_본문_합본/동화본문_211223수정.txt', 'r').read()
df2 = open('tale.txt', 'r').read()
df = []
sentence_num = 10
shift_num = 5

for i, sentence in enumerate(df1.split('\n\n\n')):
    story = sentence.split('\n\n')[1].split('\n')
    while len(story) > 10:
        df.append(" ".join(story[:sentence_num]))
        story = story[shift_num:]
    if story:
        df.append(" ".join(story[:sentence_num]))
for i in df2.split('\n\n'):
    story = i.split('\n')
    while len(story) > 10:
        df.append(" ".join(story[:sentence_num]))
        story = story[shift_num:]
    if story:
        df.append(" ".join(story[:sentence_num]))
len(df)

10415

In [142]:
# df 문장별로 대명사 성별만 바꾸는 정도로 불려주기
import re
from itertools import combinations

new_df = []
switch = {
    '소녀':'소년',
    '소년':'소녀',
    '할머니':'할아버지',
    '할아버지':'할머니',
    '여자':'남자',
    '남자':'여자',
    '아주머니':'아저씨',
    '아저씨':'아주머니',
    '공주':'왕자',
    '왕자':'공주'
}
pattern = re.compile("|".join(switch.keys()))

for sentence in df:
    key_list = [key for key in switch.keys() if key in sentence]
    key_combi = sum([list(combinations(key_list, i+1)) for i in range(len(key_list))], [])  # 1개만 바꾸는 것도 포함
    for combi in key_combi:
        switch_tmp = {tmp:switch[tmp] for tmp in combi}
        pattern_tmp = re.compile("|".join(switch_tmp.keys()))
        new_df.append(pattern_tmp.sub(lambda m: switch_tmp[re.escape(m.group(0))], sentence))                 
len(new_df)

5694

In [17]:
with open('./10sentence_df_gender.txt', 'w') as f:
    for sentence in df:
        f.write(sentence + '\n')
    for sentence in new_df:
        f.write(sentence + '\n')
f.close()

In [18]:
with open('./10sentence_df_gender.txt', encoding="utf-8") as f:
    lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]
len(lines)

16109

5 sentence per input (성별 switch)

In [144]:
# 문장 5개당 하나의 string으로, 2문장씩 겹쳐서 구성
df1 = open('동화_본문_합본/동화본문_211223수정.txt', 'r').read()
df2 = open('tale.txt', 'r').read()
df = []
sentence_num = 5
shift_num = 3

for i, sentence in enumerate(df1.split('\n\n\n')):
    story = sentence.split('\n\n')[1].split('\n')
    while len(story) > 5:
        df.append(" ".join(story[:sentence_num]))
        story = story[shift_num:]
    if story:
        df.append(" ".join(story[:sentence_num]))
for i in df2.split('\n\n'):
    story = i.split('\n')
    while len(story) > 5:
        df.append(" ".join(story[:sentence_num]))
        story = story[shift_num:]
    if story:
        df.append(" ".join(story[:sentence_num]))
len(df)

18075

In [145]:
# df 문장별로 대명사 성별만 바꾸는 정도로 불려주기
import re
from itertools import combinations

new_df = []
switch = {
    '소녀':'소년',
    '소년':'소녀',
    '할머니':'할아버지',
    '할아버지':'할머니',
    '여자':'남자',
    '남자':'여자',
    '아주머니':'아저씨',
    '아저씨':'아주머니',
    '공주':'왕자',
    '왕자':'공주'
}
pattern = re.compile("|".join(switch.keys()))

for sentence in df:
    key_list = [key for key in switch.keys() if key in sentence]
    key_combi = sum([list(combinations(key_list, i+1)) for i in range(len(key_list))], [])  # 1개만 바꾸는 것도 포함
    for combi in key_combi:
        switch_tmp = {tmp:switch[tmp] for tmp in combi}
        pattern_tmp = re.compile("|".join(switch_tmp.keys()))
        new_df.append(pattern_tmp.sub(lambda m: switch_tmp[re.escape(m.group(0))], sentence))                 
len(new_df)

6481

In [21]:
with open('./5sentence_df_gender.txt', 'w') as f:
    for sentence in df:
        f.write(sentence + '\n')
    for sentence in new_df:
        f.write(sentence + '\n')
f.close()

In [22]:
with open('./5sentence_df_gender.txt', encoding="utf-8") as f:
    lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]
len(lines)

24556

NER 시도

In [1]:
# from nltk import word_tokenize, pos_tag, ne_chunk
# # import nltk

# # nltk.download('averaged_perceptron_tagger')
# # nltk.download('maxent_ne_chunker')
# # nltk.download('words')

# ner_list = []
# for sentence in df:
#     tok = pos_tag(word_tokenize(sentence))
#     ner = ne_chunk(tok)
#     ner_list.append(ner)
# ner_list

In [2]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

# import spacy
# from collections import Counter
# import en_core_web_sm 

# nlp = en_core_web_sm.load()

# for sentence in df:
#     doc = nlp(sentence)
#     for entity in doc.ents:
#         print(f'{entity.text:12} \t {entity.label_}')

색깔 스위칭 애매한 line hand-write

**'붉은'** : 17/60 (입술처럼 붉은, 열이 많이 나는구나. 얼굴이 온통 붉은 걸 보니, 붉은 노을 | 얼굴에 붉은 기, 붉은 포도주, 피처럼 붉은, 혈색이 붉은, 붉은 피) : **ok** \
'푸른' : 44/88 (푸른 바다, 푸른 나뭇잎, 바다는 마치 하늘처럼 푸른색, 푸른 풀, 서슬 퍼런, ...) : can't \
**'노란'** : 4/108 (노란 단풍, 노란 달빛, 노란 은행잎 | ) : **ok** \
**'빨간'** : 16/266 (새빨간, 빨간 단풍, 볼이 빨간, 털이 빨간 거야. 그래서 이름도 빨강이야 | 새빨간, 새빨간 피, 빨간 사과) : **ok** \
'파란' : 24/62 (새파란, 파란 강물, 파란 하늘, 파란 잔디, 파란 바다, 파란 새싹, 파란 풀잎, ) : can't \
'누런' : 6/10 (누런 금덩어리, 싯누런 흙탕물, 누런 황소) : can't \
'하얀' : 75/196 (까만 건 글자요, 하얀 건 종이, 하얀 눈, 새하얀 종이, 하얀 쌀알, 하얀 눈, 새하얀 구름, 하얀 초승달, 하얀 백조, ...) : can't \

**'빨강'** : 3/116 (해님이 분홍 빛에서 빨강 빛으로 변하면서, 털이 빨간 거야. 그래서 이름도 빨강이야 | 빨강 뭔가가 튀어나와 뭔지 보니 어머나 이건 진짜 잘 익은 딸기) **ok** \
**'파랑'** : 9/76 (파랑새 | 파랑새) **ok** \
**'노랑'** : 0/50 ( | ) **ok** \
'초록' : 23/212 (초록색 풀, 산이 온통 초록색, 초록색 풀, 초록색 이끼, 초록빛 풀밭, 초록색 나뭇잎, 초록 숲, 초록색 잎사귀 | 상추가 너무도 싱싱하고 초록색, 초록색 이파리, 초록색 잎, 초록의 잡목 숲, 초록색 양배추, 초록색 잎사귀) **적지만 can't** \
**'주황'** : /32 ( | ) **ok**


※ 스위칭 대상 : '붉은', '노란', '빨간' | '빨강', '파랑', '노랑', '주황' \
※ '색깔들의 숨은 이야기' : 스위칭X

10 sentence per input (성별 + 색깔 switch)

In [78]:
# 문장 10개당 하나의 string으로, 5문장씩 겹쳐서 구성
df1 = open('동화_본문_합본/동화본문_211223수정.txt', 'r').read()
df2 = open('tale.txt', 'r').read()
df = []
sentence_num = 10
shift_num = 5
non_switch_title = ['\n색깔들의 숨은 이야기']
non_switch_df = []

for i, sentence in enumerate(df1.split('\n\n\n')):
    if sentence.split('\n\n')[0] in non_switch_title:
        story = sentence.split('\n\n')[1].split('\n')
        while len(story) > 10:
            non_switch_df.append(" ".join(story[:sentence_num]))
            story = story[shift_num:]
        if story:
            non_switch_df.append(" ".join(story[:sentence_num]))
        continue
    story = sentence.split('\n\n')[1].split('\n')
    while len(story) > 10:
        df.append(" ".join(story[:sentence_num]))
        story = story[shift_num:]
    if story:
        df.append(" ".join(story[:sentence_num]))
for i in df2.split('\n\n'):
    story = i.split('\n')
    while len(story) > 10:
        df.append(" ".join(story[:sentence_num]))
        story = story[shift_num:]
    if story:
        df.append(" ".join(story[:sentence_num]))
len(df)

10391

In [79]:
len(non_switch_df)

24

In [80]:
# df 문장별로 대명사 성별, 색깔 바꿔서 불려주기
import re
from itertools import combinations, product

new_df = []
switch_keys = ['소녀', '소년', '할머니', '할아버지', '여자', '남자','아주머니', '아저씨', '공주', '왕자',
               '붉은', '노란', '빨간', '빨강', '파랑', '노랑', '주황']
# genders = ['소녀', '소년', '할머니', '할아버지', '여자', '남자','아주머니', '아저씨', '공주', '왕자']
colors = {
    '붉은':['노란','빨간'],
    '노란':['붉은','빨간'],
    '빨간':['붉은','노란'],
    '빨강':['파랑','노랑','주황'],
    '파랑':['빨강','노랑','주황'],
    '노랑':['빨강','파랑','주황'],
    '주황':['빨강','파랑','노랑']
}
colors_combi = list(product(colors['붉은'], colors['노란'], colors['빨간'],
                            colors['빨강'], colors['파랑'], colors['노랑'], colors['주황']))

for sentence in df:
    key_list = [key for key in switch_keys if key in sentence]
    key_combi = sum([list(combinations(key_list, i+1)) for i in range(2,len(key_list),1)], [])  # 3개 이상 변경만 포함
    new_sentence = sentence[:]
    for combi in key_combi:
        for color in colors_combi:
            switch = {
                '소녀':'소년',
                '소년':'소녀',
                '할머니':'할아버지',
                '할아버지':'할머니',
                '여자':'남자',
                '남자':'여자',
                '아주머니':'아저씨',
                '아저씨':'아주머니',
                '공주':'왕자',
                '왕자':'공주',

                '붉은':color[0],
                '노란':color[1],
                '빨간':color[2],
                '빨강':color[3],
                '파랑':color[4],
                '노랑':color[5],
                '주황':color[6]
            }
            switch_tmp = {tmp:switch[tmp] for tmp in combi}
            pattern_tmp = re.compile("|".join(switch_tmp.keys()))
#             if not all(i in genders for i in combi):
            if new_sentence != pattern_tmp.sub(lambda m: switch_tmp[re.escape(m.group(0))], sentence):
                new_sentence = pattern_tmp.sub(lambda m: switch_tmp[re.escape(m.group(0))], sentence)
                new_df.append(new_sentence)
len(new_df)

19766

In [82]:
new_df.extend(non_switch_df)

In [83]:
len(new_df)

19790

In [76]:
with open('./10sentence_df_gender+color.txt', 'w') as f:
    for sentence in df:
        f.write(sentence + '\n')
    for sentence in new_df:
        f.write(sentence + '\n')
f.close()

In [84]:
with open('./10sentence_df_gender+color.txt', encoding="utf-8") as f:
    lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]
len(lines)

30181